In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

###########################

# Data Overview

Importa la base de datos y responde a las siguientes preguntas

In [ ]:
filename="./Data/World Alcohol Consumption Dataset.csv"
alcohol_df=pd.read_csv(filename) #lee el fichero

1 - ¿Cuantas filas y columnas tiene la base de datos?

In [ ]:
alcohol_df.shape #devuelve el numero de filas (100) y columnas (5)

2 - ¿Cuántas variables numéricas hay? ¿y categóricas?

In [ ]:
alcohol_df.info() # hay dos variables numéricas:Year y Display value, y 3 categóricas: Region, Country, and Type

3 - ¿Cuál es el último año incluído en los datos?, ¿y el primero?

In [ ]:
alcohol_df.describe() #el primer año incluído es 1984 y el último 1989

In [ ]:
#También podemos sacarlos directamente con estas funciones: .min() .max() .mean() .std() .meadian() .count()
print(alcohol_df.min())
#el valor mínimo en la columna año es 1984
print(alcohol_df.max())
#el último año es 1989

4 - ¿Cómo se distribuye el Display value?

In [ ]:
alcohol_df.hist() #hist nos permite hacer los histogramas de las variables numericas directamente


###########################
# Data subsets and query

Usando la base de datos de pokemons, responde a las siguientes preguntas

In [ ]:
filename="./Data/pokemon_clean.csv"
pokemon_df=pd.read_csv(filename,index_col=0)

1. Encuentra los pokemons legendarios de tipo hielo (con seleccion booleana)

In [ ]:
seleccion=pokemon_df["Type 1"]=="Ice"
pokemon_df[seleccion & pokemon_df["Legendary"]]

2. Encuentra los pokemons que tengan una velocidad entre 80 y 100 y que sean de tipo 1 "Grass" (usando seleccion booleana)

In [ ]:
seleccion1=pokemon_df["Speed"].between(80,100)
seleccion2=pokemon_df["Type 1"] == "Grass"
pokemon_df[seleccion1 & seleccion2]

3. Encuentra los pokemons cuyo nombre termine por "saur"  con "HP" mayor que 30 (seleccion booleana)

In [ ]:
seleccion1=pokemon_df["Name"].str.endswith("saur")
seleccion2=pokemon_df["HP"]>30
pokemon_df[seleccion1 & seleccion2]

4. Encuentra los pokemons que tienen una defensa mayor que 1.5 veces su ataque

In [ ]:
seleccion=pokemon_df["Defense"]>pokemon_df["Attack"]*1.5
pokemon_df[seleccion]

5. Encuentra todos los pokemons que tienen "Mega " en el nombre, y guarda la selección como una columna

In [ ]:
seleccion=pokemon_df["Name"].str.contains("Mega ")
pokemon_df.loc[:,"Mega"]=seleccion
pokemon_df

##########################

Usando la base de datos del señor de los anillos, responde a estas preguntas

In [ ]:
filename="./Data/WordsByCharacter.csv"
words_df=pd.read_csv(filename)
multi_df = words_df.set_index(['Film', 'Chapter', 'Race', 'Character']).sort_index()
multi_df

1. ¿Qué personajes hablan en el primer capítulo de  “The Fellowship of the Ring”? (responde con .loc)

In [ ]:
multi_df.loc[('The Fellowship Of The Ring', '01: Prologue'), :] #como estos índices son los primeros podemos directamente seleccionar el libro y el capitulo que queremos

2. ¿Cuales son los tres primeros elfos en hablar en “The Fellowship of the Ring”? (responder con .loc)

In [ ]:
multi_df.loc[('The Fellowship Of The Ring',slice(None),'Elf'), :].head(3) #aqui debemos saltarnos "Chapter" porque NO SABEMOS en qué acpitulo hablan, hay que ponerlos todos

3. ¿Cuánto hablan Saruman y Gandalf en “The Two Towers”? (responde con .loc)

In [ ]:
multi_df.loc[('The Two Towers',slice(None),slice(None),['Gandalf','Saruman']), :] # De nuevo nos saltamos los indices chapter y race, yaq solo tenemos filtro en libro y personaje

# Limpiando Datos

In [ ]:
filename="./Data/sales_order.csv"
sales_df=pd.read_csv(filename,index_col=0)
display(sales_df)

1. Write a Pandas program to detect missing values of a given DataFrame. Display True or False

In [ ]:
print(sales_df.isna())

2. Write a Pandas program to identify the column(s) of a given DataFrame which have at least one missing value.

In [ ]:
print(sales_df.isna().any())

3. Write a Pandas program to keep the rows with at least 2 NaN values in a given DataFrame.

In [ ]:
display(sales_df)
result = sales_df.dropna(thresh=2)
display(result)

# Agrupando valores 

1. Agrupa los pokemons por su generación y si son o no "Mega" (usa groupby), y obten el numéro de pokemons (usa .count()) que caen dentro de esos casos. Despoués usa seaborn para obtener un gráfico de barras en las que las categorias sean la generacion y el hue si son o no megas

In [ ]:
Mega_df=pokemon_df.groupby(["Generation","Mega"]).count()["#"].reset_index() #agrupamos según dos variables, y contamos los casos. Nos quedamos solo con 1 columna. 
# Como obtenemos una df multiindex tenemos que hacer .reset_index() para usarla en seaborn
sns.barplot(x="Generation",y="#", hue="Mega", data=Mega_df)
plt.show()

2. Encuentra si el ataque total de los poquemons Legendarios es diferente de los poquemos normales (usa sns.boxplot o sns.stripplot)

In [ ]:
pokemon_df["Total_Atk"]=pokemon_df["Attack"]+pokemon_df["Sp. Atk"] #guardar la suma de ataques en nueva columna
sns.stripplot(x='Legendary',y='Total_Atk',data=pokemon_df)
sns.boxplot(x='Legendary',y='Total_Atk',data=pokemon_df)
plt.show()
#Así podemos verlos pero no podemos saber si son realmente diferentes o no. Usemos statannot

In [ ]:
df= pokemon_df
df["Legendary_str"]= df['Legendary'].map({True: 'True', False: 'False'}) #statannot no trabaja con booleans, los convertimos en strings
x = "Legendary_str"
y = "Total_Atk"
columns = list(df[x].unique())
f, ax = plt.subplots(1, 1, figsize=(6,4))
bplot=sns.boxplot(data=df, x=x, y=y, ax=ax)
sns.stripplot(data=df, x=x, y=y, ax=ax, color='black')
test_results = add_stat_annotation(bplot, data=df, x=x, y=y, #order=order,
                                   box_pairs=[("False", "True")],#, ("Thur", "Sat"), ("Fri", "Sun")],
                                   test='t-test_ind', text_format='star',
                                   loc='outside', verbose=0)

plt.tight_layout()
plt.show()
#test_results

3. Los métodos ded agregación funcionan también en df indexadas Volviendo a la del LOTR ¿Cuánto habla Isildur en total en todas las películas? (usa .xs)

In [ ]:
multi_df.xs('Isildur', level='Character').sum() #solo necesitamos filtar al personaje Isildur, y sumar todo

4. Sin embargo funcionan mejor para cuando buscamos algo concreto. Encuentra el número de palabras que habla cada una de las razas (es más sencillo si reseteas el index y agrupas por raza)

In [ ]:
print(multi_df.reset_index().groupby("Race").sum().sort_values(by="Words",ascending=False))

#Sin embargo el número concreto de palabras que dicen los elfos es más corto con:
print("\nLos elfos dicen %s palabras" % multi_df.xs('Elf', level='Race').sum()["Words"]) 

5. ¿Cuanta cantidad de diálogo tienen las diferentes razas de media? (usa un barplot)

In [ ]:
from numpy import median
from numpy import mean
sns.barplot(data=words_df, x="Race", y="Words", estimator=mean)
plt.show()

5. Encuentra el valor medio, la varianza y el número de elementos del ataque total de cada tipo 1 de pokemon, y guardalo en unas nuevas variables

In [ ]:
#obtener el ataque total
pokemon_df["Total_Atk"]=pokemon_df["Attack"]+pokemon_df["Sp. Atk"]
#encontrar los valores medio y la dispersion y el numero de casos por grupo
attack_mean=pokemon_df.groupby(by="Type 1").mean(numeric_only=True)["Total_Atk"]
attack_std=pokemon_df.groupby(by="Type 1").std(numeric_only=True)["Total_Atk"]
attack_n=pokemon_df.groupby(by="Type 1").count()["Total_Atk"]
attack_mean

# Correlaciones

1. ¿Existe correlación entre la generación y el número de pokemons mega?

In [ ]:
pokemon_df["Mega_str"]= pokemon_df['Mega'].map({True: 'True', False: 'False'}) #statannot no trabaja con booleans, los convertimos en stringsM
Mega_by_generation=pokemon_df.groupby(["Mega_str","Generation"]).count()["#"].to_frame()#.reset_index()
Mega_by_generation.xs("True", level="Mega_str").reset_index().corr()

2. ¿Existe correlacion entre el numero total de palabras y las razas en LOTR?

In [ ]:
from pandas import factorize #vamos a tener que factorizar!
words_by_race=multi_df.reset_index().groupby("Race").sum().sort_values(by="Words",ascending=False).reset_index()
display(words_by_race)
labels, categories = factorize(words_by_race["Race"])
words_by_race["Race_labels"] = labels
print("corr:",abs(words_by_race["Words"].corr(words_by_race["Race_labels"])))
sns.scatterplot(x="Race_labels", y="Words", data=words_by_race)
plt.show()

# Combinando DataFrames

1. Combina las tres series obtenidas antes (ataque total medio, varianza y nº de casos) en una nueva dataframe, ordena los tipos en funcion de su ataque, y plotea el resultado usando un gráfico de barras

In [ ]:
total_atack_df=pd.concat([attack_mean, attack_std,attack_n], axis=1) #si son más de dos es más sencillo con concat que permite pegar una lsita de df segun flas o columnas
total_atack_df.columns=["Mean","Std","N"]
total_atack_df.sort_values("Mean",ascending=False,inplace=True)
print(total_atack_df)
#f, ax= plt.errorbar(total_atack_df.index, 'Mean', yerr='Std',data=total_atack_df)
plt.bar(total_atack_df.index, total_atack_df["Mean"], yerr=total_atack_df["Std"], align='center', alpha=0.5, ecolor='black', capsize=10)
plt.xticks(rotation = 90)
plt.tight_layout()
plt.show()

1.b Haz un gráfico de barras del ataque total de los pokemons usando sns.barplot 

In [ ]:
sns.barplot(x="Type 1", y="Total_Atk", data=pokemon_df)#,order=total_atack_df.index) #¿Cómo puedo ordenarlos de forma decreciente? usando la df de arriba
plt.xticks(rotation = 90)
plt.show()
#Seaborn hace muchos cálculos por ti, pero para ordenar y ponerlo bonito vas a necesitar tener manejo de los groupby, medidas agregadas y sort

# Re-shaping DataFrames

1. ¿Qué Hobbit habla más en cada película, y a lo largo de las tres películas? (responde con pivot y .loc)


Primero obtenemos una tabla con los valores de "Words" agregados para cada personaje en cada película. Para eso vamos a expandir la columna Film, y como función vamos a calcular la suma de las palabras. Mantendremos como indices tanto la raza como los personajes.

In [ ]:
pivoted = multi_df.pivot_table(
                         #values="Words", #si no especifico genera un multiindex con varios valores
                         index = ['Race','Character'] ,#los valores que se quedan como index
                         columns = 'Film', #lo que vamos a convertir en columnas
                         aggfunc = 'sum', #como agregamos los valores, aqui sumando
                         fill_value = 0).sort_index()
order = [('Words', 'The Fellowship Of The Ring'),
         ('Words', 'The Two Towers'),
         ('Words', 'The Return Of The King')]
pivoted.loc[:,('Words', 'All Films')]=pivoted.sum(axis=1)

display(pivoted)

Ahora queremos ver en especial solo cuales son los hobbits que hablan mas, es decir, nos interesa una raza en particular

In [ ]:
pivoted.loc['Hobbit'].sort_values([('Words', 'All Films')],ascending=False)

2. Continuemos con Pivot y ahora  vamos a obtner la defensa media en función del tipo 1 y si es o no legendario

In [ ]:
pokemon_df.pivot_table(
                         values=["Defense","HP"], #si no especifico genera un multiindex con varios valores
                         index = ['Type 1','Legendary'] ,#los valores que se quedan como index
                         #columns = 'Film', #lo que vamos a convertir en columnas
                         aggfunc = 'sum', #como agregamos los valores, aqui sumando
                         fill_value = 0).sort_index()